Датасет: https://www.kaggle.com/datasets/rajyellow46/wine-quality

# Качество вина

Набор данных был загружен из репозитория машинного обучения UCI.

Два набора данных относятся к красному и белому вариантам португальского вина «Виньо Верде». Ссылка [Кортез и др., 2009]. Из соображений конфиденциальности и логистики доступны только физико-химические (входные) и сенсорные (выходные) переменные (например, отсутствуют данные о сортах винограда, марке вина, отпускной цене вина и т. д.).

Эти наборы данных можно рассматривать как задачи классификации или регрессии. Классы упорядочены и не сбалансированы (например, нормальных вин намного больше, чем отличных или плохих). Алгоритмы обнаружения выбросов можно использовать для обнаружения нескольких отличных или плохих вин. Кроме того, мы не уверены, все ли входные переменные актуальны. Поэтому было бы интересно протестировать методы выбора функций.

Два набора данных были объединены, и несколько значений были удалены случайным образом.

**Цель исследования:** восстановить удаленные данные.

**Ход исследования:**
1. Обзор данных
2. Предобработка данных
3. Проверка гипотез

## 1. Обзор данных

In [10]:
#импортируем библиотеки
import os
import pandas as pd

In [16]:
#загрузка файла с kaggle
def load_data_from_kaggle():
    data_path = 'wine-quality-data'

    if not os.path.exists(data_path):
        !kaggle datasets download -d rajyellow46/wine-quality        
        with zipfile.ZipFile('wine-quality.zip', 'r') as zipp:
            zipp.extractall(data_path)

    train_path = "\\".join([data_path, os.listdir(data_path)[0]])
    
    return pd.read_csv(train_path)

In [17]:
#выводим на экран первые 10 строк
df = load_data_from_kaggle()
df.head(10)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,white,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,white,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [18]:
#выводим общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


В таблице 13 столбцов. Тип данных в столбцах - `object`, `float64`, `int64`. Количество значений в столбцах различается, что подтверждает наличие пропусков.

Согласно документации к данным:

* `type` — тип вина;
* `fixed acidity` — фиксированная кислотность;
* `volatile acidity` — летучая кислотность;
* `citric acid` — лимонная кислота;
* `residual sugar` — остаточный сахар;
* `chlorides` - хлориды;
* `free sulfur dioxide` — свободный диоксид серы;
* `total sulfur dioxide` — общий диоксид серы;
* `density` — плотность;
* `pH` — pH;
* `sulphates` — сульфаты;
* `alcohol` — спирт;
* `quality` — качество (оценка от 0 до 10);

**Вывод:**

Обнаружены пропуски данных в следующих столбцах: `fixed acidity`, `volatile acidity`, `citric acid`, `residual sugar`, `chlorides`, `pH`, `sulphates`, `sulphates`.

Также переименуем названия столбцов в стиль snake_case.

## 2. Предобработка данных

### 2.1 Стиль заголовков

In [29]:
df.rename(columns={elem: '_'.join(elem.split(' ')) for elem in list(df.columns)}, inplace=True)
df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


### 2.2 Пропуски значений

In [30]:
#считаем кол-во прропусков
df.isna().sum()

type                     0
fixed_acidity           10
volatile_acidity         8
citric_acid              3
residual_sugar           2
chlorides                2
free_sulfur_dioxide      0
total_sulfur_dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64